In [ ]:
import pickle
from extract_consecutive_screenshots import get_pickle_name, load_objects, load_snapshot_data, analyze_run
from visualization import get_connection, get_gta_image_jpg_dir
from os import path
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib import patches
import numpy as np
# JPEG_DIR = get_gta_image_jpg_dir()
JPEG_DIR = 'D:/GTAV_extraction_output/rgb-jpeg'

In [ ]:
run_id = 52
analyze_run(run_id)

In [ ]:
cars = load_objects(run_id)
len(cars)

In [ ]:
def get_pos_diffs(snapshots):
    pos_diffs = []
    positions = [snap['position'] for snap in snaps]
    for i in range(1, len(snaps)):
        diff = np.array(positions[i]) - np.array(positions[i - 1])
        pos_diffs.append(diff)
    return pos_diffs

def transform_to_relative(diff, cam):
    base = np.array([
        [cam[0], cam[1]],
        [cam[1], -cam[0]]
    ])
    return np.matmul(base, diff[0:2])

In [ ]:
# todo: udělat i time diff mezi snapshoty a vydělit jím velikost diff_rel, aby to bylo znormované na sekundu
for key, car in cars.items():
    snaps = car['snapshots']
    pos_diffs = get_pos_diffs(snaps)
    for i in range(len(pos_diffs)):
        snap = snaps[i]
        diff = pos_diffs[i]
        diff_rel = transform_to_relative(diff, snap['cam_direction'][0:2])
        
        snap['diff'] = diff
        snap['diff_relative'] = diff_rel